In [1]:
import numpy as np
from numpy.linalg import inv, det
import matplotlib.pyplot as plt
from copy import copy
import random
import math
%matplotlib inline 

## Работа с множествами игроков

In [2]:
def generate_sets(size, players = None):
    def to_bin_vector(value):
        return np.array([(value >> i) & 1 for i in range(size)])
    
    if players is None:
        players = np.arange(size)
        
    banned_ids = np.delete(np.arange(size), players)
    
    return [tuple(to_bin_vector(n)) for n in range(2**size) if (to_bin_vector(n)[banned_ids] == 0).all()]

def is_intersects(lhs, rhs):
    return np.minimum(np.array(lhs) == 1, np.array(rhs) == 1).any()

def indexes_array(bitvector):
    return np.arange(bitvector.shape[0])[bitvector]

## Супераддитивность

In [3]:
def is_supper_additive(size, prices):
    all_combinations = generate_sets(size)

    for combination in all_combinations:
        comb_price = prices[combination]

        player_ids = indexes_array(np.array(combination) == 1)

        subcombinations = generate_sets(size, player_ids)
        
        for i in range(len(subcombinations)):
            for j in range(i + 1, len(subcombinations)):
                lhs = tuple(subcombinations[i])
                rhs = tuple(subcombinations[j])
                
                if is_intersects(lhs, rhs):
                    continue
                
                if comb_price < prices[lhs] + prices[rhs]:
                    return (False, combination, [lhs, rhs])

    return (True, )

## Выпуклось

In [4]:
def bit_where(func, lhs, rhs):
    return np.where(func(lhs == 1, rhs == 1), 
                    np.ones(lhs.shape[0], dtype = int),
                    np.zeros(lhs.shape[0], dtype = int))

def is_convex(size, prices):
    all_combinations = generate_sets(size)

    for i in range(len(all_combinations)):
        for j in range(i + 1, len(all_combinations)):
            lhs = np.array(all_combinations[i])
            rhs = np.array(all_combinations[j])
            
            intersection = tuple(bit_where(np.minimum, lhs, rhs))
            union = tuple(bit_where(np.maximum, lhs, rhs))

            if prices[union] + prices[intersection] < prices[tuple(lhs)] + prices[tuple(rhs)]:
                return (False, [intersection, union], [tuple(lhs), tuple(rhs)])
                
    return (True, )

## Шэпли

In [5]:
def shaply_vector(size, prices, player):
    sets = generate_sets(size)
    
    summary = 0
    
    for S in sets:
        if S[player] != 0:
            summary += math.factorial(np.sum(S) - 1) * math.factorial(size - np.sum(S)) \
                * (prices[S] - prices[tuple(np.where(np.arange(len(S)) == player, 0, S))])
            
    return (1 / math.factorial(size)) * summary

def is_group_rational(size, prices):
    individual_sum = np.sum([shaply_vector(size, prices, i) for i in range(size)])
    
    return abs(individual_sum - prices[tuple(np.ones(size))]) < 0.00001
    
def is_self_rational(size, prices):
    for i in range(size):
        key = np.zeros(size)
        key[i] = 1
        
        if shaply_vector(size, prices, i) < prices[tuple(key)]:
            return False
        
    return True

## Решение варианта

In [6]:
prices_var15 = {
    (0, 0, 0, 0) : 0,
    (1, 0, 0, 0) : 2,
    (0, 1, 0, 0) : 3,
    (1, 1, 0, 0) : 6,
    (0, 0, 1, 0) : 2,
    (1, 0, 1, 0) : 5,
    (0, 1, 1, 0) : 6,
    (1, 1, 1, 0) : 9,
    (0, 0, 0, 1) : 4,
    (1, 0, 0, 1) : 7,
    (0, 1, 0, 1) : 8,
    (1, 1, 0, 1) : 11,
    (0, 0, 1, 1) : 8,
    (1, 0, 1, 1) : 10,
    (0, 1, 1, 1) : 12,
    (1, 1, 1, 1) : 14,
}

In [7]:
super_additive = is_supper_additive(4, prices_var15)

if super_additive[0]:
    print('Game is super additive')
else:
    print('Game is not super additive')
    print(super_additive[1], '<', super_additive[2])

Game is super additive


In [8]:
conv = is_convex(4, prices_var15)
    
if conv[0]:
    print('Game is convex')
else:
    print('Game is not convex')
    
    players1 = ['v({' + ', '.join(str(j) for j in indexes_array(np.array(i) == 1)) + '})' for i in conv[1]]
    players2 = ['v({' + ', '.join(str(j) for j in indexes_array(np.array(i) == 1)) + '})' for i in conv[2]]
    
    print(' + '.join(players1), '<', ' + '.join(players2))
    
    prices1 = [str(prices_var15[i]) for i in conv[1]]
    prices2 = [str(prices_var15[i]) for i in conv[2]]
    
    print(' + '.join(prices1), '<', ' + '.join(prices2))

Game is not convex
v({1}) + v({0, 1, 2, 3}) < v({0, 1}) + v({1, 2, 3})
3 + 14 < 6 + 12


In [9]:
vec = [shaply_vector(4, prices_var15, i) for i in range(4)]

print(np.round(vec, 3))
print(round(np.sum(vec), 3))

[2.417 3.75  2.917 4.917]
14.0


In [10]:
print('Self rational:', is_self_rational(4, prices_var15))

Self rational: True


In [11]:
print('Group rational:', is_group_rational(4, prices_var15))

Group rational: True
